In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import itertools as it
import lingam
from LOVO_via_parent_adjustment import edge_might_exist, lovo_comparison
from extended_pywhy_graphs import ExtendedADMG
from pandarallel import pandarallel
import pandas as pd
import random

### LOVO for Grace Data and "ground truth" graph

In [ ]:
data_grace = pd.read_csv('real_world_data/Grace_et_al_plot_level.csv')
data_grace = data_grace[["ln.site.rich", "ln.site.totmass", "ln.site.prod", "ln.prich", "ln.pshade", "ln.ptotmass", "ln.pprod", "SoilSuitability"]]

true_bidirected_edges=[('ln.site.rich', 'ln.site.prod'), ('ln.site.rich', 'ln.site.totmass')],
true_directed_edges=[('ln.site.prod', 'ln.site.totmass'),
    ('ln.site.totmass', 'ln.site.rich'),
    ('ln.site.rich', 'ln.site.prod'),
    ('SoilSuitability', 'ln.prich'),
    ('SoilSuitability', 'ln.site.rich'),
    ('ln.site.rich', 'ln.prich'),
    ('ln.site.prod', 'ln.pprod'),
    ('ln.site.totmass', 'ln.ptotmass'),
    ('ln.prich', 'ln.ptotmass'),
    ('ln.ptotmass', 'ln.pshade'),
    ('ln.pshade', 'ln.prich'),
    ('ln.prich', 'ln.pprod')]

dict = {name: i for i, name in enumerate(data_grace.columns)}
true_bidirected_edges = [(dict[i], dict[j]) for (i, j) in true_bidirected_edges]    
true_directed_edges = [(dict[i], dict[j]) for (i, j) in true_directed_edges]

true_graph_grace = ExtendedADMG(incoming_nodes = set(range(8)), E_bidir = true_bidirected_edges, E_dir = true_directed_edges)
true_graphs_without_i = {i: true_graph_grace.project_to_GX(i) for i in range(8)}
all_pairs = list(it.combinations(range(8), 2))
data_val = data_grace.copy()
data_val.columns = range(8)
lovo_errors, baseline_errors = zip(*[lovo_comparison(X, Y, true_graphs_without_i[Y], true_graphs_without_i[X], data_val, 'Lemma_2') for (X, Y) in all_pairs])
np.nanmean(np.array(lovo_errors)), np.nanmean(np.array(baseline_errors))

### LOVO for Sachs Data and "ground truth" graph

In [ ]:
from cdt.data import load_dataset
s_data, s_graph_nx = load_dataset("sachs")
import networkx as nx
nx.draw(s_graph_nx, with_labels=True)

In [ ]:
# s_graph has different ordering of variables compared to s_data
s_data.rename(columns={name: i for i, name in enumerate(s_graph_nx.nodes)}, inplace=True)

adj_matrix = nx.to_numpy_array(s_graph_nx).T
true_graph = ExtendedADMG.from_RCD_adjacency_matrix(adj_matrix)
true_graphs_without_i = {i: true_graph.project_to_GX(i) for i in range(8)}
all_pairs = list(it.combinations(range(8), 2))
data_val = s_data[:int(7466/2)]
lovo_errors, baseline_errors = zip(*[lovo_comparison(X, Y, true_graphs_without_i[Y], true_graphs_without_i[X], data_val, 'Lemma_2') for (X, Y) in all_pairs])
np.nanmean(np.array(lovo_errors)), np.nanmean(np.array(baseline_errors))

### Fit using DirectLiNGAM, RCD

In [ ]:
def compute_CV_errors(data, setting):
    n, nr_nodes = data.shape
    n_learn = int(n/2) 
    if n_learn < 1000:
        data_learn, data_val = data, data
    else:
        data_learn, data_val = data[:n_learn], data[n_learn:] 
    
    learned_graphs_without_i = {}
    for X in range(nr_nodes):
        data_without_X = np.delete(data_learn, X, axis=1)
        model = lingam.RCD() if setting == 'RCD' else lingam.DirectLiNGAM()
        model.fit(data_without_X)
        adj_matrix = model.adjacency_matrix_
        # Insert zeros such that indices (and therefore the names of the nodes in the graph) match the original indices in data
        adj_matrix_expanded = np.insert(adj_matrix, X, 0, axis=0)  
        adj_matrix_expanded = np.insert(adj_matrix_expanded, X, 0, axis=1)
        learned_graphs_without_i[X] = ExtendedADMG.from_RCD_adjacency_matrix(adj_matrix_expanded)

    all_pairs = list(it.combinations(range(nr_nodes), 2))
    data_val.columns = range(nr_nodes)
    lovo_errors, baseline_errors = zip(*[lovo_comparison(X, Y, learned_graphs_without_i[Y], learned_graphs_without_i[X], data_val.copy(), 'Lemma_2' if setting == 'RCD' else 'Lemma_4') for (X, Y) in all_pairs])
    return lovo_errors, baseline_errors

for data in [s_data, data_grace]:
    data = (data - data.mean()) / data.std()
    for setting in ['DirectLiNGAM', 'RCD']:
        lovo_errors, baseline_errors = compute_CV_errors(data, setting)
        print(f'For {setting}, LOVO loss is {np.nanmean(np.array(lovo_errors))} and baseline loss is {np.nanmean(np.array(baseline_errors))}')